# Model 1 - XBoost algorithm: Baseline to compare with the performance of the neural networks

In [1]:
import pandas as pd
import tflearn as tf
import numpy as np
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tflearn.data_utils import to_categorical, pad_sequences
from tflearn.layers.recurrent import bidirectional_rnn, BasicLSTMCell
import pickle
import os
from tensorflow.contrib.tensorboard.plugins import projector

from preprocessing_merged import *
from itertools import combinations_with_replacement, permutations
import csv


In [0]:
#######################

In [2]:
# LOAD DIRECT THE SET, SKIP LATER 
#os.chdir('/Users/laurieprelot/Google Drive/Machine Learning/project 2/Data Sets')

In [3]:
#USE WHOLE PREPROCESSED DATA SET LATER
import pickle
trainX,trainY,testX,testY,X_test,num_vocab,features = pickle.load(open('train_test_prep_expand_full_2','rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'train_test_prep_expand_full_2'

In [0]:
testY=testY[:,1] # Because of the 'to_categorical function'
trainY=trainY[:,1]

In [0]:
# from numpy import argmax
# a=argmax(testY, axis=1)

In [0]:
#XBOOST START
import xgboost as xgb
from xgboost import  DMatrix
from xgboost.sklearn import XGBClassifier

from sklearn import model_selection, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
from sklearn.pipeline import make_pipeline, FeatureUnion, Pipeline
from sklearn.model_selection import RandomizedSearchCV

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [0]:
# Reduce data set IF NEEDED
a=1000
trainX_foo=trainX[1:a]
trainY_foo=trainY[1:a]
testX_foo=testX[1:a]
testY_foo=testY[1:a]
X_test=X_test[1:a]

In [0]:
#Helper 1:Tune Parameters
predictors='tweet'
target='smile'

'''TO CHANGE 
1. cv_folds
2. n_iter!! : int, default=10 Number of parameter settings that are sampled'''
def TuneParam(xgb1, trainX, trainY, param_test1, cv_folds=3, early_stopping_rounds=50):
    '''This function splits the training and the validation test in training and Validation Test and 
    performs a random gridsearch on the input gridof parameters
    Inputs: Parameters Xboost, trainX, trainY, grid parameters to optimoze
    Returns: The best Parameters and the best score 
    
    Inputs:
    trainX, trainY: training set and its labels
    param_test1: Grid of parameters or their distribution if continous
    cv_folds: Cross validation folds for tuning the parameters
    
    Outputs: 
    search1.grid_scores_: Results grid 
    gsearch1.best_params_: Dictionary with the optimal choice of parameters for tuning
    gsearch1.best_score_: Best Log loss achieved 
    '''
    xgb_param = xgb1.get_xgb_params()
    # Do Cross validation and Grid Search. Use the xgb parameters and the test parameter grid provide
    gsearch1 = RandomizedSearchCV(estimator = XGBClassifier(learning_rate =xgb_param['learning_rate'],\
                                                      n_estimators=xgb_param['n_estimators'],\
                                                      max_depth=xgb_param['max_depth'],\
                                                      min_child_weight=xgb_param['min_child_weight'],\
                                                      gamma=xgb_param['gamma'],\
                                                      subsample=xgb_param['subsample'],\
                                                      colsample_bytree=xgb_param['colsample_bytree'],\
                                                      objective= xgb_param['objective'],\
                                                      nthread=xgb_param['nthread'],\
                                                      scale_pos_weight=xgb_param['scale_pos_weight'],\
                                                      seed=xgb_param['seed']), \
                            param_distributions= param_test1, scoring='neg_log_loss',\
                                  n_jobs=4,iid=False, cv=cv_folds, n_iter=2) #CHANGE CV HERE
    
    gsearch1.fit(trainX,trainY)
    print( gsearch1.best_params_, gsearch1.best_score_)
    
#         xgb_param = xgb1.get_xgb_params()
#         xgtrain = DMatrix(X, label=Y)
#         cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=xgb_param['n_estimators'], nfold=cv_folds,\
#                                     metrics='auc', early_stopping_rounds=early_stopping_rounds)
#         xgb1.set_params(n_estimators=cvresult.shape[0])
    return gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_




In [0]:
#Helper 2 : Get the error of the model 
def modelfit(xgb1, trainX, trainY, testX, testY, early_stopping_rounds=50):
    
    '''The function performs fits the model on a training set and does the predicton on the testing set. 
    Input: 
    trainX, trainY, testX, testY:....
    
    Outputs:
    predictions, predprob, acc: ....
    
     '''
    #Fit the algorithm on the data
    xgb1.fit(trainX, trainY ,eval_metric='logloss')
        
    #Predict on testing set set:
    predictions = xgb1.predict(testX)
    predprob = xgb1.predict_proba(testX)[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(testY, predictions))
    acc="Accuracy : %.4g" % metrics.accuracy_score(testY, predictions)
    #auc="AUC Score (Train): %f" % metrics.roc_auc_score(testY, predprob)                                            

    return predictions, predprob, acc

In [0]:
# Set of initial parameters 
xgb1 = XGBClassifier(
 learning_rate =0.1,#Typical final values to be used: 0.01-0.2
 n_estimators=100, #'''CHANGE'''
 max_depth=5,       #Typical values: 3-10
 min_child_weight=1, 
 gamma=0,           #minimum loss reduction required to make a split.
 subsample=0.8,     #Typical values: 0.5-1
 colsample_bytree=0.8,#Typical values: 0.5-1
 objective= 'binary:logistic', # loss function to be minimized. 
 nthread=4,
 scale_pos_weight=1, #A value greater than 1 should be used in case of high class imbalance 
 seed=27)

Best_param={}

In [0]:
#  Optimization of parameters on train set #0

#Tune -> n_estimators
param_test = {'n_estimators':range(20,201,10)}
gsearch1, best_params_i, best_score_i=TuneParam(xgb1, trainX_foo, trainY_foo, param_test)
Best_param.update(best_params_i)

#Update
xgb1 = XGBClassifier(
 learning_rate =0.1, #Typical final values to be used: 0.01-0.2
 n_estimators=Best_param['n_estimators'],
 max_depth=5,       #Typical values: 3-10
 min_child_weight=1,
 gamma=0,           #minimum loss reduction required to make a split.
 subsample=0.8,     #Typical values: 0.5-1
 colsample_bytree=0.8,#Typical values: 0.5-1
 objective= 'binary:logistic', # loss function to be minimized. 
 nthread=4,
 scale_pos_weight=1, #A value greater than 0 should be used in case of high class imbalance 
 seed=27)

KeyboardInterrupt: 

In [0]:
#  Optimization of parameters on train set #1

#Tune -> max_depth, min_child_weight
param_test = {
 'max_depth':np.arange(3,10,2),
 'min_child_weight':np.arange(1,17,2)
}

gsearch1, best_params_i, best_score_i=TuneParam(xgb1, trainX_foo, trainY_foo, param_test)
Best_param.update(best_params_i)

#Update 
xgb1 = XGBClassifier(
 learning_rate =0.1, #Typical final values to be used: 0.01-0.2
 n_estimators=Best_param['n_estimators'],
 max_depth=Best_param['max_depth'], #Typical values: 3-10
 min_child_weight=Best_param['min_child_weight'],
 gamma=0,#minimum loss reduction required to make a split.
 subsample=0.8,#Typical values: 0.5-1
 colsample_bytree=0.8,#Typical values: 0.5-1
 objective= 'binary:logistic', # loss function to be minimized. 
 nthread=4,
 scale_pos_weight=1,#A value greater than 0 should be used in case of high class imbalance 
 seed=27)


KeyboardInterrupt: 

In [0]:
#  Optimization of parameters on train set #1b

#Tune -> max_depth, min_child_weight
param_test = {
 'max_depth':np.arange(max(Best_param['max_depth']-2,3),min(Best_param['max_depth']+2,10),1),
 'min_child_weight':np.arange(max(Best_param['min_child_weight']-2,1),min(Best_param['min_child_weight']+2,17),1)
}

gsearch1, best_params_i, best_score_i=TuneParam(xgb1, trainX_foo, trainY_foo, param_test)
Best_param.update(best_params_i)

#Update 
xgb1 = XGBClassifier(
 learning_rate =0.1, #Typical final values to be used: 0.01-0.2
 n_estimators=Best_param['n_estimators'],
 max_depth=Best_param['max_depth'], #Typical values: 3-10
 min_child_weight=Best_param['min_child_weight'],
 gamma=0,#minimum loss reduction required to make a split.
 subsample=0.8,#Typical values: 0.5-1
 colsample_bytree=0.8,#Typical values: 0.5-1
 objective= 'binary:logistic', # loss function to be minimized. 
 nthread=4,
 scale_pos_weight=1,#A value greater than 0 should be used in case of high class imbalance 
 seed=27)

In [0]:
#  Optimization of parameters on train set #2

#Tune -> Gamma
param_test = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch1, best_params_i, best_score_i= TuneParam(xgb1, trainX_foo, trainY_foo, param_test)
Best_param.update(best_params_i)

#Update 
xgb1 = XGBClassifier(
 learning_rate =0.1, #Typical final values to be used: 0.01-0.2
 n_estimators=Best_param['n_estimators'],
 max_depth=Best_param['max_depth'], #Typical values: 3-10
 min_child_weight=Best_param['min_child_weight'],
 gamma=Best_param['gamma'],#minimum loss reduction required to make a split.
 subsample=0.8,#Typical values: 0.5-1
 colsample_bytree=0.8,#Typical values: 0.5-1
 objective= 'binary:logistic', # loss function to be minimized. 
 nthread=4,
 scale_pos_weight=1,#A value greater than 0 should be used in case of high class imbalance 
 seed=27)

KeyboardInterrupt: 

In [0]:
#  Optimization of parameters on train set #3

#Tune -> subsample and colsample_bytree

param_test= {
 'subsample':[i/10.0 for i in range(5,10)],
 'colsample_bytree':[i/10.0 for i in range(5,10)]
}
gsearch1, best_params_i, best_score_i= TuneParam(xgb1, trainX_foo, trainY_foo, param_test)
Best_param.update(best_params_i)

#Update 
xgb1 = XGBClassifier(
 learning_rate =0.1, #Typical final values to be used: 0.01-0.2
 n_estimators=Best_param['n_estimators'],
 max_depth=Best_param['max_depth'], #Typical values: 3-10
 min_child_weight=Best_param['min_child_weight'],
 gamma=Best_param['gamma'],#minimum loss reduction required to make a split.
 subsample=Best_param['subsample'],#Typical values: 0.5-1
 colsample_bytree=Best_param['colsample_bytree'],#Typical values: 0.5-1
 objective= 'binary:logistic', # loss function to be minimized. 
 nthread=4,
 scale_pos_weight=1,#A value greater than 0 should be used in case of high class imbalance 
 seed=27)


In [0]:
#  Optimization of parameters on train set #3b

#Tune -> subsample and colsample_bytree

param_test= {
 'subsample':[i/100.0 for i in range((int(Best_param['subsample']*100))-5,\
                                     (int(Best_param['subsample']*100))+10,5)],
 'colsample_bytree':[i/100.0 for i in range((int(Best_param['colsample_bytree']*100))-5,\
                                            (int(Best_param['colsample_bytree']*100))+10,5)]
}
gsearch1, best_params_i, best_score_i= TuneParam(xgb1, trainX_foo, trainY_foo, param_test)
Best_param.update(best_params_i)

#Update 
xgb1 = XGBClassifier(
 learning_rate =0.1, #Typical final values to be used: 0.01-0.2
 n_estimators=Best_param['n_estimators'],
 max_depth=Best_param['max_depth'], #Typical values: 3-10
 min_child_weight=Best_param['min_child_weight'],
 gamma=Best_param['gamma'],#minimum loss reduction required to make a split.
 subsample=Best_param['subsample'],#Typical values: 0.5-1
 colsample_bytree=Best_param['colsample_bytree'],#Typical values: 0.5-1
 objective= 'binary:logistic', # loss function to be minimized. 
 nthread=4,
 scale_pos_weight=1,#A value greater than 0 should be used in case of high class imbalance 
 seed=27)


In [0]:
#  Optimization of parameters on train set #4

#Tune -> Regularization Parameters
param_test = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch1, best_params_i, best_score_i= TuneParam(xgb1, trainX_foo, trainY_foo, param_test)
Best_param.update(best_params_i)


#Update 
xgb1 = XGBClassifier(
 learning_rate =0.1, #Typical final values to be used: 0.01-0.2
 n_estimators=Best_param['n_estimators'],
 max_depth=Best_param['max_depth'], #Typical values: 3-10
 min_child_weight=Best_param['min_child_weight'],
 gamma=Best_param['gamma'],#minimum loss reduction required to make a split.
 subsample=Best_param['subsample'],#Typical values: 0.5-1
 colsample_bytree=Best_param['colsample_bytree'],#Typical values: 0.5-1
 objective= 'binary:logistic', # loss function to be minimized. 
 nthread=4,
 scale_pos_weight=1,#A value greater than 0 should be used in case of high class imbalance 
 seed=27,
 reg_alpha=Best_param['reg_alpha'])

KeyboardInterrupt: 

In [0]:
#  Optimization of parameters on train set #5

#Tune -> Learning Rate
param_test = {
 'learning_rate': np.logspace(0.01, 0.6, 7, endpoint=True)/10
}
gsearch1, best_params_i, best_score_i= TuneParam(xgb1, trainX_foo, trainY_foo, param_test)
Best_param.update(best_params_i)


#Update 
xgb1 = XGBClassifier(
 learning_rate =Best_param['learning_rate'], #Typical final values to be used: 0.01-0.2
 n_estimators=Best_param['n_estimators'],
 max_depth=Best_param['max_depth'], #Typical values: 3-10
 min_child_weight=Best_param['min_child_weight'],
 gamma=Best_param['gamma'],#minimum loss reduction required to make a split.
 subsample=Best_param['subsample'],#Typical values: 0.5-1
 colsample_bytree=Best_param['colsample_bytree'],#Typical values: 0.5-1
 objective= 'binary:logistic', # loss function to be minimized. 
 nthread=4,
 scale_pos_weight=1,#A value greater than 0 should be used in case of high class imbalance 
 seed=27,
 reg_alpha=Best_param['reg_alpha'])

In [0]:
#IF NOT TOO HEAVY 
'''lower the learning rate and add more trees.'''
# xgb4 = XGBClassifier(
#  learning_rate =0.01,
#  n_estimators=5000,
#  max_depth=Best_param['max_depth'], #Typical values: 3-10
#  min_child_weight=Best_param['max_child_weight'],
#  gamma=Best_param['gamma'],#minimum loss reduction required to make a split.
#  subsample=Best_param['subsample'],#Typical values: 0.5-1
#  colsample_bytree=Best_param['colsample_bytree'],#Typical values: 0.5-1
#  objective= 'binary:logistic', # loss function to be minimized. 
#  nthread=4,
#  scale_pos_weight=1, #A value greater than 0 should be used in case of high class imbalance 
#  seed=27,
#  reg_alpha=Best_param['reg_alpha'])

In [0]:
# Get Final Error of the Model
predictions_test, predprob_test, accuracy_glob=modelfit(xgb1, trainX_foo, trainY_foo, testX_foo, testY_foo)
    


Model Report
Accuracy : 0.4444


In [0]:
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})

In [0]:
ids=..
#Predict XTest 
predictions_Resuts = xgb1.predict(X_test)
predprob_Results = xgb1.predict_proba(X_test)[:,1]
create_csv_submission(list(ids), list(predictions_test), 'Prediction_Xboost')  

In [0]:
import time
import numpy as np
now = time.time() #(seconds)
a = []
for j in range(2):
  for s in range(10):
    a.append(np.random.random())
then = now
print(time.time() - then)

0.0006878376007080078
